### LCEL Deepdive

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

app_dir = os.path.join(os.getcwd(), "app")
load_dotenv(os.path.join(app_dir, ".env"))

In [ ]:
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

In [ ]:
prompt.invoke({"topic": "ice cream"})

In [ ]:
from langchain_core.messages.human import HumanMessage

messages = [HumanMessage(content='tell me a short joke about ice cream')]
model.invoke(messages)

### What is this "|" in Python?

In [ ]:
from abc import ABC, abstractmethod

class CRunnable(ABC):
    def __init__(self):
        self.next = None

    @abstractmethod
    def process(self, data):
        """
        This method must be implemented by subclasses to define
        data processing behavior.
        """
        pass

    def invoke(self, data):
        processed_data = self.process(data)
        if self.next is not None:
            return self.next.invoke(processed_data)
        return processed_data

    def __or__(self, other):
        return CRunnableSequence(self, other)

class CRunnableSequence(CRunnable):
    def __init__(self, first, second):
        super().__init__()
        self.first = first
        self.second = second

    def process(self, data):
        return data

    def invoke(self, data):
        first_result = self.first.invoke(data)
        return self.second.invoke(first_result)



In [ ]:
class AddTen(CRunnable):
    def process(self, data):
        print("AddTen: ", data)
        return data + 10

class MultiplyByTwo(CRunnable):
    def process(self, data):
        print("Multiply by 2: ", data)
        return data * 2

class ConvertToString(CRunnable):
    def process(self, data):
        print("Convert to string: ", data)
        return f"Result: {data}"

In [ ]:
a = AddTen()
b = MultiplyByTwo()
c = ConvertToString()

chain = a | b | c

In [ ]:
result = chain.invoke(10)
print(result)

### Runnables from LangChain

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel

In [23]:
chain = RunnablePassthrough() | RunnablePassthrough () | RunnablePassthrough ()
chain.invoke("hello")

'hello'

In [24]:
def input_to_upper(input: str):
    output = input.upper()
    return output

In [25]:
chain = RunnablePassthrough() | RunnableLambda(input_to_upper) | RunnablePassthrough()
chain.invoke("hello")

'HELLO'

In [26]:
chain = RunnableParallel({"x": RunnablePassthrough(), "y": RunnablePassthrough()})

In [27]:
chain.invoke("hello")

{'x': 'hello', 'y': 'hello'}

In [28]:
chain.invoke({"input": "hello", "input2": "goodbye"})

{'x': {'input': 'hello', 'input2': 'goodbye'},
 'y': {'input': 'hello', 'input2': 'goodbye'}}

In [29]:
chain = RunnableParallel({"x": RunnablePassthrough(), "y": lambda z: z["input2"]})

In [30]:
chain.invoke({"input": "hello", "input2": "goodbye"})

{'x': {'input': 'hello', 'input2': 'goodbye'}, 'y': 'goodbye'}

### Nested chains - now it gets more complicated!

In [31]:
def find_keys_to_uppercase(input: dict):
    output = input.get("input", "not found").upper()
    return output

In [32]:
chain = RunnableParallel({"x": RunnablePassthrough() | RunnableLambda(find_keys_to_uppercase), "y": lambda z: z["input2"]})

In [33]:
chain.invoke({"input": "hello", "input2": "goodbye"})

{'x': 'HELLO', 'y': 'goodbye'}

In [34]:
chain = RunnableParallel({"x": RunnablePassthrough()})

def assign_func(_):
    return 100

def multiply(input):
    return input * 10

In [35]:
chain.invoke({"input": "hello", "input2": "goodbye"})

{'x': {'input': 'hello', 'input2': 'goodbye'}}

In [36]:
chain = RunnableParallel({"x": RunnablePassthrough()}).assign(extra=RunnableLambda(assign_func))

In [37]:
result = chain.invoke({"input": "hello", "input2": "goodbye"})
print(result)

{'x': {'input': 'hello', 'input2': 'goodbye'}, 'extra': 100}


### Combine multiple chains

In [38]:
def extractor(input: dict):
    return input.get("extra", "Key not found")

def cupper(upper: str):
    return str(upper).upper()

new_chain = RunnableLambda(extractor) | RunnableLambda(cupper)

In [41]:
new_chain.invoke({"extra": "test"})

'TEST'

In [42]:
final_chain = chain | new_chain
final_chain.invoke({"input": "hello", "input2": "goodbye"})

'100'